Method 1: Encode library into latent space
Method 2: Generate library from latent vector (options: which vector to use)
Method 3: Analyze library statistics (diversity)

In [ ]:
paths = ['../working/cmx_latent.pkl', '../working/origin_latent.pkl', '../working/wt_latent.pkl']


# code for plotting training, origin, and WT
# A MESS
# FOR SOME REASON THE FIRST FILE HAS DIFFERENT KEYS, I THINK BECAUSE I RAN SAMEER'S CODE?
# flag = True
for i, path in enumerate(paths):
#     if flag:
#         flag = False
#         file = open(path, 'rb')
#         data = pickle.load(file)
#     #     print(data.keys())
#         means = data['background_latent']
#     #     means = [foo[0].tolist() for foo in latents]
#         xs = [mean[0] for mean in means]
#         ys = [mean[1] for mean in means]
#     else:
    file = open(path, 'rb')
    data = pickle.load(file)
#     print(data.keys())
    latents = data['latent']
    means = [foo[0].tolist() for foo in latents]
    xs = [mean[0] for mean in means]
    ys = [mean[1] for mean in means]

    if i == 2:
        plt.scatter(xs, ys, s=.5, color='red')
    else:
        plt.scatter(xs, ys, s=.5)

# code to plot repeated 2d training set
# for i in range(5):
#     path = '../working/2d_%d.pkl'%i
#     file = open(path, 'rb')
#     data = pickle.load(file)
# #     print(data.keys())
#     latents = data['latent']
#     means = [foo[0].tolist() for foo in latents]
#     xs = [mean[0] for mean in means]
#     ys = [mean[1] for mean in means]

#     plt.scatter(xs, ys, s=.5)    

plt.legend(['Training set', 'Generated from origin', 'Generated from WT'])
plt.xlabel('Latent Dim 1')
plt.ylabel('Latent Dim 2')

In [ ]:
config = Config('../config_1000_15.yaml')
dataset = utils.get_dataset_from_config(config)
seqs = list(dataset)
seqs = torch.stack([row[0] for row in seqs])
seqs = seqs.reshape(seqs.size()[0], -1, 21)
model = utils.load_model_from_config(config)

In [ ]:
latent = pickle.load(open(config.latent_fullpath, 'rb'))['latent']
means = [row[0] for row in latent]
log_vars = [row[1] for row in latent]

In [ ]:
def sample_from_vector(means, log_vars, sample_size):
    with torch.no_grad():
        outputs = []
        for i in range(sample_size):
            encoding = model.reparameterize(means, log_vars)
            output = model.decoder(encoding)
            outputs.append(output)
        outputs = torch.stack(outputs)
        recons = utils.softmax(outputs)
        # before reshape: sample_size x one_hot_encoding
        recons = recons.reshape(sample_size, -1, 21)
        # after reshape: sample_size x seq_length x amino_acids
    return recons

In [ ]:
def explore_latent_vector(means, log_vars, sample_size=100):
    all_recons = {}

    with torch.no_grad():

        # all variation (normal sampling)
        # torch.stack([]) makes this a 2D tensor
        free_recons = sample_from_vector(means, log_vars, sample_size)
        all_recons['free'] = free_recons.reshape(1, *free_recons.size())

        # all fixed (no variation)
        fixed_outputs = []
        fixed_log_vars = torch.full(log_vars.size(), -float('inf')) # bc e^(-inf)=0
        fixed_recons = sample_from_vector(means, fixed_log_vars, sample_size)
        all_recons['fixed'] = fixed_recons.reshape(1, *fixed_recons.size())

        # one variation, others fixed
        single_free = []
        for free_var in range(len(log_vars)):
            single_free_log_vars = torch.full(log_vars.size(), -float('inf'))
            single_free_log_vars[free_var] = log_vars[free_var]
            single_free.append(sample_from_vector(means, single_free_log_vars, sample_size))
        all_recons['one_free']=torch.stack(single_free)
        
        # one fixed, others variation
        single_fixed = []
        for fixed_var in range(len(log_vars)):
            single_fixed_log_vars = log_vars.clone()
            single_fixed_log_vars[fixed_var] = -float('inf')
            single_fixed.append(sample_from_vector(means, single_fixed_log_vars, sample_size))
        all_recons['one_fixed'] = torch.stack(single_fixed)
            
    return all_recons

In [ ]:
def explore_latent_origin(latent_size, sample_size=100):
    all_recons = {}
    # mean 0 and variance 1
    means = torch.zeros(latent_size)
    log_vars = torch.ones(latent_size)
    return explore_latent_vector(means, log_vars, sample_size=sample_size)

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

def freqs_to_heatmap(freqs, adjustment=None, label='adjustment', xlabel=None):
    %matplotlib inline
    
    to_plot = []
    adjusted = freqs
    if not adjustment is None:
        adjusted = freqs-adjustment
        to_plot.extend(adjustment)
    to_plot.extend(adjusted)
    to_plot = torch.stack(to_plot)
    
    # current state of matrix: [#latents+stuff x #positions x amino_acids ]
    plottable = to_plot.transpose(1, 2)
    # current state of matrix: [#latents+stuff x amino_acids x #positions]
    plottable = plottable.reshape(-1, plottable.size()[-1])
    # current state of matrix: [(#latents*21 amino acids) x #positions]


    figure = plt.figure(figsize=(15,15), dpi=250)
    ax = plt.axes()
    ax.set_yticks(range(0, len(plottable), 21))
    ax.set_yticklabels([label, *range(0, len(plottable)-2)])
    if not xlabel==None:
        ax.set_xticks(range(0, len(plottable[0])))
        ax.set_xticklabels(ss_dataset,fontsize=3, color='red')
    ax.tick_params(colors='red')
    heatmap = ax.imshow(plottable)
    divider = make_axes_locatable(ax)
    
    ax.set_ylabel('Variation type', color='red')
    ax.set_xlabel('Position', color='red')

    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = figure.colorbar(heatmap, cax=cax)
    cbar.ax.tick_params(colors='red')
    
    return figure
    

In [ ]:
input_wt = seqs[0]
trimmed_wt = [aa for aa in input_wt if not aa[-1]==0]
print(len(trimmed_wt))
print(len(wt_ss))

# input_3vdh = seqs[-1]
aligned_ss = []
i = 0
for aa in input_wt:
    if not aa[-1] == 0:
        aligned_ss.append(wt_ss[i][1])
        i+=1
    else:
        aligned_ss.append('-')
i = None

latent_wt = (means[0], log_vars[0])

# these are all dictionaries of various latent sampling for different starting sequences
recons_wt = explore_latent_vector(means[0], log_vars[0], sample_size=10)
# recons_3vdh = explore_latent_vector(means[-1], log_vars[-1], sample_size=10)
recons_o = explore_latent_origin(len(means[0]), sample_size = 10)



In [ ]:
# this is a plot of structure vs sequence for WT

figure = plt.figure(figsize=(15,15), dpi=250)
ax = plt.axes()
ax.imshow(input_wt.transpose(0,1))

In [ ]:
freqs_wt = examine_model.seqs_to_freqs_dict(recons_wt)
# freqs_3vdh = examine_model.seqs_to_freqs_dict(recons_3vdh)
freqs_o = examine_model.seqs_to_freqs_dict(recons_o)

In [ ]:
one_free_wt_fig = freqs_to_heatmap(freqs_wt['one_free'], adjustment=freqs_wt['fixed'],
                                   label='no variation', xlabel=ss_dataset)

In [ ]:
one_fixed_wt_fig = freqs_to_heatmap(freqs_wt['one_fixed'], adjustment=freqs_wt['free'], label='all variation')

In [ ]:
def freqs_to_diversity(freqs_dict):
    # input tensor dimensions: latent_group x position x amino_acids
    # this is Shannon's index. IDK if it's a good metric, though
    divs = {}
    for key, freqs in freqs_dict.items():
        print(key)
        divs[key] = torch.tensor([[-torch.sum(pos[~(pos==0)]*torch.log(pos[~(pos==0)])) for pos in seq] for seq in freqs])
    return divs

def plot_diversity(div):
    fig = plt.figure(figsize=(30, 30), dpi=250)
    ax = plt.axes()
    ax.set_yticks(range(0,len(div)))
    ax.set_yticklabels(['Free', 'Fixed', *range(0, len(div)-2)], color='red')
    
    ax.tick_params(axis='y', labelsize=3)
    heatmap = ax.imshow(div)
    divider = make_axes_locatable(ax)

    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = fig.colorbar(heatmap, cax=cax)
    cbar.ax.tick_params(colors='red')

In [ ]:
divs_wt = freqs_to_diversity(freqs_wt)
# here I change divs_wt to a list for plotting
# it is jank. I will fix later
plottable = []
for val in divs_wt.values():
    for div in val:
        plottable.append(div)

plottable = torch.stack(plottable)
divs_wt_fig = plot_diversity(plottable)

# Converting WT diversity to 3vdh diversity
## Uses sampling based on WT seq

In [ ]:
# wt seq aligned with dataset
wt_oh_gaps = input_wt.reshape(1, *input_wt.size())
wt_seq_gaps = utils.one_hots_to_proteins(wt_oh_gaps)
wt_divs_gaps = torch.cat(list(divs_wt.values()))
labels = {'free':slice(0,1), 'fixed':slice(1,2), 'one_free':slice(2,17), 'one_fixed':slice(17,32)}

# wt seq with no gaps
wt_seq_divs = utils.remove_gaps(wt_seq_gaps, wt_divs_gaps)
wt_seq = wt_seq_divs[0]
wt_divs = wt_seq_divs[1:]

# wt seq aligned with 3vdh
wt_seq_align = np.array(wt_aln) # taken from cell 3
wt_divs_align = np.full((len(wt_divs), len(wt_seq_align)), np.nan)
base_i = 0
for (align_i, aa) in enumerate(wt_seq_align):
    if aa == '-':
        pass
    elif wt_seq[base_i] == aa:
        wt_divs_align[:, align_i] = wt_divs[:, base_i]
        base_i += 1
    else:
        print('uh oh the sequences don\'t match!')
        print(base_i, align_i)
        print(wt_seq[base_i], aa)
        break
i = None

# final product: wt_divs_align